<a href="https://colab.research.google.com/github/agrimwood/medphys_helper/blob/main/MPC_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Imports for JupyterLite
%pip install -q ipywidgets
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import pandas as pd
import glob
from matplotlib import pyplot as plt


In [ ]:
# read all csv files to dataframe
lst = []
for f in glob.glob('/content/*.csv'):
  df = pd.read_csv(f)
  lst.append(df)
df = pd.concat(lst,ignore_index=True)

# simple interactive plot of MPC data
pd.set_option('copy_on_write', True)
@interact(TestParameter=df.columns[6::2], linac=[x for x in df['SN'].unique() if str(x) != 'nan'], energy=[x for x in df['Energies'].unique() if str(x) != 'nan'])
def dataPlot(TestParameter,linac,energy):
    dfplt = df[(df['SN']==linac) & (df['Energies']==energy)]
    dfplt['Testdate'] = pd.to_datetime(dfplt['Testdate'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
    dfplt[TestParameter] = pd.to_numeric(df[TestParameter], errors='coerce')
    dfplt = dfplt[['Testdate',TestParameter]].dropna()
    dfplt.plot(x='Testdate',y=TestParameter, xlabel="Date", ylabel=TestParameter, title = energy+": "+TestParameter, legend=False)
    plt.show()
    return